## First, install the necessary libraries:

In [1]:
!pip install opencv-python face_recognition numpy pandas matplotlib seaborn scikit-learn

## Setup Environment

In [1]:
# Import necessary libraries
import cv2
import face_recognition
import numpy as np
import pandas as pd
import datetime
import os
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns

# Set up directories for images and data
if not os.path.exists('employee_faces'):
    os.makedirs('employee_faces')

if not os.path.exists('attendance_logs'):
    os.makedirs('attendance_logs')


## Register Employees

In [ ]:
import cv2
import numpy as np
from IPython.display import display, clear_output
from PIL import Image
import os
import time

def register_employee():
    # Get employee details from input
    employee_id = input("Enter Employee ID: ")
    employee_name = input("Enter Employee Name: ")
    
    # Create the directory to save employee images if it doesn't exist
    if not os.path.exists('employee_faces'):
        os.makedirs('employee_faces')
    
    # Open the webcam
    video_capture = cv2.VideoCapture(0)  # 0 is usually the default webcam
    
    if not video_capture.isOpened():
        print("Error: Could not open webcam.")
        return
    
    print(f"Capturing face for {employee_name} (ID: {employee_id})... Image will be saved in 5 seconds.")
    
    start_time = time.time()  # Record the start time
    
    while True:
        # Capture frame-by-frame
        ret, frame = video_capture.read()
        
        if not ret:
            print("Failed to capture image.")
            break
        
        # Convert the frame to RGB (OpenCV uses BGR by default)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Convert the frame to a PIL image for display in Jupyter Notebook
        img_pil = Image.fromarray(frame_rgb)
        
        # Display the frame in the notebook
        clear_output(wait=True)  # Clear previous frames
        display(img_pil)  # Display current frame
        
        # Check if 5 seconds have passed
        if time.time() - start_time > 5:
            # Save the current frame as the employee's image
            image_path = f"employee_faces/{employee_id}_{employee_name}.jpg"
            cv2.imwrite(image_path, frame)  # Save the image in the 'employee_faces' directory
            print(f"Image saved at {image_path}")
            break
    
    # Release the webcam
    video_capture.release()

# Call the function to start registration
register_employee()


## Facial Recognition for Check-In/Check-Out

In [27]:
import cv2
import face_recognition
import numpy as np
import os
from datetime import datetime

# Initialize video capture (0 for the default camera)
video_capture = cv2.VideoCapture(0)

# Load known faces and their names
known_faces = []
known_names = []

# Load your known faces and names from your dataset (update this part with your logic)
def load_known_faces(known_faces_directory):
    for filename in os.listdir(known_faces_directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image = face_recognition.load_image_file(os.path.join(known_faces_directory, filename))
            face_encoding = face_recognition.face_encodings(image)[0]
            known_faces.append(face_encoding)
            known_names.append(os.path.splitext(filename)[0])  # Use the filename as the name

load_known_faces('employee_faces')  # Update the path

# Function to log attendance
def log_attendance(name):
    with open("attendance.csv", "a") as f:
        now = datetime.now()
        f.write(f"{name},{now.strftime('%Y-%m-%d %H:%M:%S')}\n")

def check_in_check_out():
    while True:
        # Capture frame
        ret, frame = video_capture.read()
        if not ret:
            print("Failed to capture frame. Exiting...")
            break
        
        # Convert the image from BGR to RGB
        rgb_small_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Resize frame for faster face recognition processing
        rgb_small_frame = cv2.resize(rgb_small_frame, (0, 0), fx=0.25, fy=0.25)

        # Find all face locations and encodings
        face_locations = face_recognition.face_locations(rgb_small_frame)
        print("Detected face locations:", face_locations)  # Debug info

        if face_locations:  # Check if any faces were found
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
            for face_encoding, face_location in zip(face_encodings, face_locations):
                # Check if the face matches any known face
                matches = face_recognition.compare_faces(known_faces, face_encoding)
                
                name = "Unknown"
                if True in matches:
                    first_match_index = matches.index(True)
                    name = known_names[first_match_index]
                    print(f"Recognized: {name}")  # Debug info
                    log_attendance(name)  # Log attendance for recognized user
                else:
                    print("No match found.")  # Debug info

                # Draw a rectangle around the face
                top, right, bottom, left = face_location
                top *= 4
                right *= 4
                bottom *= 4
                left *= 4

                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)

        # Display the resulting image
        cv2.imshow('Video', frame)

        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the capture and destroy windows
    video_capture.release()
    cv2.destroyAllWindows()

# Call the check-in/check-out function to start the system
check_in_check_out()


Detected face locations: [(46, 109, 82, 73)]
Recognized: 101_Rahul Kumar
Detected face locations: [(46, 105, 82, 69)]
Recognized: 101_Rahul Kumar
Detected face locations: [(46, 105, 82, 69)]
Recognized: 101_Rahul Kumar
Detected face locations: [(46, 105, 82, 69)]
Recognized: 101_Rahul Kumar
Detected face locations: [(46, 105, 82, 69)]
Recognized: 101_Rahul Kumar
Detected face locations: [(46, 105, 82, 69)]
Recognized: 101_Rahul Kumar
Detected face locations: [(46, 105, 82, 69)]
Recognized: 101_Rahul Kumar
Detected face locations: [(46, 105, 82, 69)]
Recognized: 101_Rahul Kumar
Detected face locations: [(46, 105, 82, 69)]
Recognized: 101_Rahul Kumar
Detected face locations: [(46, 105, 82, 69)]
Recognized: 101_Rahul Kumar
Detected face locations: [(46, 105, 82, 69)]
Recognized: 101_Rahul Kumar
Detected face locations: [(46, 105, 82, 69)]
Recognized: 101_Rahul Kumar
Detected face locations: [(46, 105, 82, 69)]
Recognized: 101_Rahul Kumar
Detected face locations: [(46, 105, 82, 69)]
Recogn

## Spoof Detection

In [33]:
import cv2
import face_recognition
import numpy as np
import dlib
from IPython.display import display, clear_output
import time

# Load the pre-trained detector for facial landmarks
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

def is_spoof(frame):
    """
    Simple spoof detection based on eye blink or liveness detection.
    You can extend this with more advanced techniques.
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    
    for face in faces:
        # Get the landmarks for the face
        landmarks = predictor(gray, face)
        
        # Extract the coordinates of the eyes (landmark points 36 to 41 and 42 to 47)
        left_eye = [landmarks.part(i) for i in range(36, 42)]
        right_eye = [landmarks.part(i) for i in range(42, 48)]
        
        # Convert to numpy arrays for further processing
        left_eye_pts = np.array([[p.x, p.y] for p in left_eye])
        right_eye_pts = np.array([[p.x, p.y] for p in right_eye])

        # Compute the eye aspect ratio (EAR) to check for blink detection
        left_EAR = eye_aspect_ratio(left_eye_pts)
        right_EAR = eye_aspect_ratio(right_eye_pts)

        # Average eye aspect ratio
        avg_EAR = (left_EAR + right_EAR) / 2.0

        # EAR threshold to detect blink
        EAR_THRESHOLD = 0.25  # Adjust based on your testing
        if avg_EAR < EAR_THRESHOLD:
            print("Blink detected - Liveness confirmed!")
            return False  # It's not a spoof (live person)
    
    return True  # If no face or blink is detected, consider it as spoof

def eye_aspect_ratio(eye):
    """
    Compute the Eye Aspect Ratio (EAR) which can be used to detect blink.
    EAR = (|p2 - p6| + |p3 - p5|) / (2 * |p1 - p4|)
    """
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    EAR = (A + B) / (2.0 * C)
    return EAR

# Example integration into facial recognition system
def check_in_check_out():
    video_capture = cv2.VideoCapture(0)

    while True:
        ret, frame = video_capture.read()
        if not ret:
            print("Failed to capture frame")
            break

        # Resize frame for faster processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        rgb_small_frame = small_frame[:, :, ::-1]  # Convert from BGR to RGB

        # Detect face locations
        face_locations = face_recognition.face_locations(rgb_small_frame, model="hog")
        
        # Detect face encodings based on the detected face locations
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
        
        # Spoof Detection
        if is_spoof(frame):
            print("Possible Spoof Detected!")
        else:
            print("Live person detected")

        # Display the frame within the Jupyter Notebook
        clear_output(wait=True)
        _, buffer = cv2.imencode('.jpg', frame)
        display_image = buffer.tobytes()
        display(Image(data=display_image))

        time.sleep(0.03)  # Small delay to allow for real-time updates

        # Break the loop after displaying a few frames for testing purposes
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam after finishing
    video_capture.release()
    cv2.destroyAllWindows()

check_in_check_out()


RuntimeError: Unable to open shape_predictor_68_face_landmarks.dat

## Log Attendance

In [34]:
def log_attendance(name):
    current_time = datetime.datetime.now()
    log_entry = {'Name': name, 'Time': current_time}

    log_df = pd.DataFrame([log_entry])
    log_df.to_csv(f'attendance_logs/{current_time.date()}.csv', mode='a', header=not os.path.exists(f'attendance_logs/{current_time.date()}.csv'), index=False)

    print(f"Attendance logged for {name} at {current_time}")


## Shift & Overtime Tracking

In [36]:
def calculate_overtime(attendance_df):
    attendance_df['Time'] = pd.to_datetime(attendance_df['Time'])
    attendance_df['Date'] = attendance_df['Time'].dt.date
    attendance_df['Hour'] = attendance_df['Time'].dt.hour

    # Group by employee and calculate work hours per day
    work_hours = attendance_df.groupby(['Name', 'Date'])['Hour'].agg(['min', 'max'])
    work_hours['WorkHours'] = work_hours['max'] - work_hours['min']

    overtime = work_hours[work_hours['WorkHours'] > 8]  # Overtime if more than 8 hours
    return overtime


## Generate Attendance Reports

In [39]:
import pandas as pd
from datetime import datetime

# Function to calculate working hours from attendance data
def generate_attendance_report(csv_file):
    """
    Generates an attendance report based on check-in/check-out times.
    
    Parameters:
    - csv_file (str): Path to the CSV file containing attendance logs.
    
    Returns:
    - A pandas DataFrame containing the report.
    """
    # Read the attendance data from the CSV file
    attendance_data = pd.read_csv(csv_file, header=None, names=["Name", "Timestamp", "Action"])

    # Convert 'Timestamp' to datetime for easier manipulation
    attendance_data['Timestamp'] = pd.to_datetime(attendance_data['Timestamp'])

    # Separate the check-ins and check-outs
    check_in_data = attendance_data[attendance_data['Action'] == 'check-in']
    check_out_data = attendance_data[attendance_data['Action'] == 'check-out']

    # Merge the check-in and check-out data based on the 'Name'
    report = pd.merge(check_in_data, check_out_data, on='Name', suffixes=('_in', '_out'))

    # Calculate the total working hours by subtracting the check-in time from the check-out time
    report['Working Hours'] = report['Timestamp_out'] - report['Timestamp_in']

    # Optional: Convert 'Working Hours' to a numeric format (hours only)
    report['Working Hours'] = report['Working Hours'].apply(lambda x: x.total_seconds() / 3600)  # Convert to hours

    # Return the attendance report
    return report[['Name', 'Timestamp_in', 'Timestamp_out', 'Working Hours']]

# Example usage: generate report
attendance_report = generate_attendance_report('attendance.csv')

# Display the report
print(attendance_report)


Empty DataFrame
Columns: [Name, Timestamp_in, Timestamp_out, Working Hours]
Index: []


## Data Analysis & Insights

In [40]:
def analyze_trends(attendance_df):
    # Calculate absenteeism trends
    absent_days = attendance_df.groupby('Name')['Date'].nunique()

    # Plot absenteeism
    absent_days.plot(kind='bar')
    plt.title('Absenteeism Trends')
    plt.show()

# Load sample attendance data
attendance_df = pd.read_csv('attendance_logs/2024-10-02.csv')
analyze_trends(attendance_df)


FileNotFoundError: [Errno 2] No such file or directory: 'attendance_logs/2024-10-02.csv'